# Tutorial 3: Calibrating the Sediment Transport Model (STM)
The STM is the component of the EGM Framework that provides the distribution of sediment concentration over the floodplain. This model demands all outputs from the Hydrodynamic model: water depths, *h*, velocity *v* and discharge, *Q*. Its physics consider the sediment advection and settling processes (please see STM documentation *SedTrans_model.docx*, *Garcia et al. (2015)* and *Breda et al. (2020) @ HESS journal* for more details).


The STM requires two parameters:
* $U_{cd}$: Critical velocity (m/s).
* $w_{s}$: Settling velocity (m/s).


In this tutorial we will see how to run this model using some of the classes/functions found in the **egmlib** library. The main idea here is to conduct a parameter's calibration under different tidal regimes, but using the EGM Framework to connect the STM to the Hydrodynamic model and run these different cases.

    Garcia, M. L., et al. Modelling extraordinary floods and sedimentological processes in a large channel-floodplain system. International Journal of Sediment Research (2015), http://dx.doi.org/10.1016/j.ijsrc.2015.03.007
    Breda, A., Saco, P. M., Sandi, S. G., Saintilan, N., Riccardi, G., Rodriguez, J. F. (2020) Accretion, retreat and transgression of coastal wetlands experiencing sea-level rise. Hydrology and Earth System Sciences, (under reviewing)


## Dependencies
* You will need all files created with `example_domain2`.
* All files in the `Common Files` folder. This folder should be inside the folder with all Tutorials' files.

***
# STEP 0: Preparing files and loading the egmlib library
Create a folder named **tutorial03** and copy all files from **domain02** and **Common Files** to this new folder.


In the code below replace the string `lib_folder` with the path to folder where the **egmlib.py** file is and than run the cell. Just note that we should use `/` or `\\` instead of a single `\` as Python understand this last as a indicator of a special character.


The egmlib library already include the import of many other libraries, as **numpy** and **matplotlib** that will be used later in this notebook.

In [ ]:
lib_folder = 'C:/Wetlands/Tutorials'
from sys import path
path.append(lib_folder)
from egmlib import *

# STEP 1: Initialising the EGM Framework
As we saw in *Tutorial 2*, to initialise an instance of **EGMframework** class we need to provide a file with the simulation settings and the directory's path with all files to run the Hydrodynamic model.


We will create a `json` file with the following information:
- Simulation title (not usefull at all, but good to use as reference)
- Time-step for the Hydrodynamic outputs ($\Delta t$ between records)
- Values of STM parameters: Ucd and ws
- Path to the file with input water level series
- Path to the file with input sediment concentration series
- Boolean to tell if the input files has the first column as index/time-labels


Once the settings file was created, we can create an instance of the framework and initialise it!

In [ ]:
# Directory's path where the model will run
run_folder = 'C:\\Wetlands\\Tutorials\\tutorial03'

# Creating dictionary with information for EGM simulation
settings = {
    'title': 'Tutorial 3',
    'Dt_HST': 300,
    'Ucd': 0.02,
    'ws': 0.0002,
    'input_levels_file': 'C:\\Wetlands\\Tutorials\\water_levels_2.csv',
    'input_sediment_file': 'C:\\Wetlands\\Tutorials\\sediment_conc_2.csv',
    'first_column_as_index': 'yes'
}

# Saving settings in a json file
filename = os.path.join(run_folder, 'egm_setup.json')
with open(filename, 'w') as file:
    json.dump(settings, file)

# Creating a new instance of EGMframework
egm = EGMframework()
egm.initialize(filename, run_folder)

# Printing some attributes of egm
print('\nNumber of cells and links:', egm.NoC, egm.NoL)
print('Water depths boundary condition applied to:', egm.loc_HT)
print('Series for EGM simulation:')
for i in range(egm.Nstages):
    print('    series %i: %s' % (i, egm.stages[i]))

***
***
From this point on, it is import to clarify the concept: **stage**

A **stage** is a single step in the EGM simulation. It can be a single year like in a run from 2000 to 2100, or just a id/name. **Stages** are taken from the header (column labels) of input water levels CSV file, and follow the same order presented there (i.e. from left to right). Parallel to stages we have the **stage index (stix)**, which is a simple numeration of these stages (first stage = 0, second stage = 1, and so on). This one is necessary to access those datasets that are not pandas.DataFrame, like the EGM outputs which are numpy.ndarrays. 
***
***

# STEP 2: Single Hydrodynamic & Sediment Transport (HST) run
Before figure out how to "play" with each series, and other things related to running the STM, let's make a single HST run and visualize its results.


For such, we need to:
1. Update the HxT boundary condition file with the data from '1 m' series
2. Update the running settings to use Dt_HST as the time-step between records in the output files, and to run until the end of the input-time-series
3. Run the hydrodynamic model
4. Run the sediment transport model
5. Plot water depth and sediment concentration at different time steps

In [ ]:
#1. Update the HxT boundary condition file with the data from '1 m' series
# We will use the function 'boundary_depths_update()' which, based on the information provided, will update the
#file with HxT boundary conditions. We will provide:
# levels = array of DataFrame with water levels
# elevs = array with bottom elevation in all domain
# Dt_HT = time-step (delta time) between records in the input series (this information will be added to egm as a
#         new attribute)
egm.Dt_inputs = egm.levels.index[1] - egm.levels.index[0]    #the index is the series' time-stamp in seconds
print('Time-step of input series =', egm.Dt_inputs, 's')
boundary_depths_update(levels=egm.levels['1 m'], elevs=egm.z, Dt_HT=egm.Dt_inputs)

#2. Update the running settings
# Similar as we did for boundary depths, we will use 'initial_condition_update()' providing the time-step for the
#output files and the final time, which will be computed to match the time-span of the input series.

# Input series always starts at t = 0, therefore if it has 3 records, the final time is at 2*Dt_inputs
final_time = (len(egm.levels['1 m']) - 1) * egm.Dt_inputs
print('HST simulation time-span =', final_time, 's')

# The Hydrodynamic model split the simulation in 4 sub-intervals. Because of that we need to provide the time at
#the end of each sub-interval. Usually, we provide a very short duration for the first 3 sub-intervals
initial_condition_update(t_final=[60, 120, 180, final_time], Dt_output=egm.Dt_HST)

#3. Run the hydrodynamic model
t0 = datetime.now()
egm.update_hydrodynamic()
print(' > Elapsed time:', (datetime.now() - t0))

#4. Run the sediment transport model
t0 = datetime.now()
data = egm.update_sediment_transport(stage='1 m')
print(' > Elapsed time:', (datetime.now() - t0))

# 2.1 Hourly depth and ssc plot
Let's create a big picture of water depths and sediment concentration through a tidal cycle (last 12 hours)

In [ ]:
hours = np.arange(1, data.NoTS+1) * data.Dt_output / 3600
ti = [np.where(hours == hour)[0][0] for hour in range(13,25)]

fig, axes = pyplot.subplots(nrows=4, ncols=3, figsize=(12,12), dpi=100, constrained_layout=True)
for i, ax in enumerate(axes.flatten()):
    ax.plot(egm.x, data.h[ti[i],:], color='tab:blue')
    ax2 = ax.twinx()
    ax2.plot(egm.x, data.ssc[ti[i],:], color='tab:orange')
    ax.grid()
    ax.set_xlabel('distance from tidal creek (m)')
    ax.set_ylabel('water depth (m)', color='tab:blue')
    ax2.set_ylabel('sediment conc. (g/m3)', color='tab:orange')
    ax.set_ylim(-0.01, np.max(data.h)+0.01)
    ax2.set_ylim(-0.5, np.max(data.ssc)+0.5)
    ax.set_xlim(-20,610)
    ax2.set_xlim(-20,610)
    ax.text(0.98, 0.98, str('hour %i' % int(hours[ti[i]])), va='top', ha='right', transform=ax.transAxes)

pyplot.show()

# STEP 3: Sequencial HST simulations
As we have input series for different tidal ranges, we should put the model to run with these different inputs one after the other. Moreover, we didn't mind the effect of the initial conditions in the results from both models. So, what we will do now is to create a loop through the EGM stages, and for each stage make an inner loop to run the model a few times, using the final condition of one run as the initial condition for the next one.


There is another way to avoid having the initial condition affecting the results. As we are using a sinusoidal wave, we could have replicated it a few times, put the models to run and take only the last period to carry on our analysis.

In [ ]:
#Initialising an empty list to store the highFreqData object from the simulation of each tidal range
db = []

#Loop through the input series (EGM stages)
t0 = datetime.now()
for stage in egm.stages:
    
    # Initialise data as None so the functions within the inner loop will proceed as if there is no previous run
    #to update the current hydrodynamic input files. Also, as we have already defined the attributes Dt_HST and
    #Dt_inputs, we don't need to do that again (assuming they won't change).
    data = None
    print('\nSimulating stage', stage)
    
    #The 'error' will be used to address the convergence to a stable result between two consecutive runs of
    #the Hydrodynamic model.
    error, iteration = 999, 1
    
    # Inner loop to warm-up the models (fading initial condition)
    while error > 0.0005:
        #the function below update boundary depths and initial condition files. Actually, it also update the
        #surface roughness and bottom elevation. However, as we are not providing such information, they won't
        egm.initialize_hydrodynamic(stage, hfd=data)
        
        #run the hydrodynamic model
        data = egm.update_hydrodynamic(returnType='highFreqData')
        
        if iteration > 1:
            # initial condition for the STM will be taken from last time-step of previous simulation
            data.ssc0 = next_ssc0
        
        #run sediment transport model
        data = egm.update_sediment_transport(stage, hfd=data)
        next_ssc0 = data.ssc[-1,:]
        
        #computing error:
        if iteration > 1:
            error = np.max(np.abs(data.h - previous))
            print('\nIteration %i;    Maximum Abs. Error = %.4f' % (iteration, error))
        
        previous = data.h
        iteration += 1
        
    #storing the highFreqData object of current stage
    db.append(data)
print('\nElapsed time:', (datetime.now() - t0))

## 3.1 Plotting time evolution
Let's see how the water depths and sediment concentration change over time at 5 points of the floodplain

In [ ]:
# Let's pick those cells with x-coordinate at 5, 205, 405, 605 and 805 m far from the tide input creek
cells = [np.where(egm.x == dist)[0][0] for dist in [5, 205, 405, 605, 805]]

# Plotting water depths
fig, axes = pyplot.subplots(ncols=5, dpi=100, figsize=(12,4), constrained_layout=True)
for i in range(len(cells)):
    c = cells[i]
    for j in range(egm.Nstages):
        axes[i].plot(hours, db[j].h[:,c], label=egm.stages[j])
    axes[i].set_title(str('x = %i m' % egm.x[c]))
    axes[i].grid()
    axes[i].set_xlabel('Hour')
equalise_yrange(axes)
axes[-1].legend(loc='best')
fig.suptitle('Water Depth (m)')
pyplot.show()

# Plotting sediment concentration
fig, axes = pyplot.subplots(ncols=5, dpi=100, figsize=(12,4), constrained_layout=True)
for i in range(len(cells)):
    c = cells[i]
    for j in range(egm.Nstages):
        axes[i].plot(hours, db[j].ssc[:,c], label=egm.stages[j])
    axes[i].set_title(str('x = %i m' % egm.x[c]))
    axes[i].grid()
    axes[i].set_xlabel('Hour')
equalise_yrange(axes)
axes[-1].legend(loc='best')
fig.suptitle('Suspended Sediment Concentration (g/m$^3$)')
pyplot.show()

# 3.2 Plotting profile of weighted-average SSC
To check if the adopted parameters are adequate, we plot the average sediment concentration profile. This average is weighted by the water levels, so that we have the sum of all sediment mass divide by the total volume of water:

$SSC_{avg} = \frac{\sum{SSC_{t} \times h_{t}}}{\sum{h_{t}}}$

We usually include the SSC profile from the bathtub model in the plot to have a idea how both models compare. The bathtub uses a linear relationship between sediment concentration and mean-depth-below-high-tide ($D$), which can be easily calculated from a sinusoidal series. In the code below we will plot the average sediment concentration from the STM using solid lines and from the Bathtub approach using dotted ones.

In [ ]:
# list to store sediment concentration from bathtub model (it will be used again later)
Cbt = []

#computing average concentration and plotting it
fig, ax = pyplot.subplots(dpi=100, figsize=(8,4), constrained_layout=True)
for i in range(egm.Nstages):
    #bathtub
    h = bathtub_depths(np.array(egm.levels[egm.stages[i]]), egm.z)
    d = np.max(h,axis=0)
    Cbt.append(bathtub_ssc(d, 37))
    #STM
    C = np.average(db[i].ssc, axis=0, weights=db[i].h)
    #plotting
    p = ax.plot(egm.x[2:], C[2:], label=egm.stages[i])
    ax.plot(egm.x[2:], Cbt[i][2:], ':', color=p[-1].get_color())
    
ax.grid()
ax.legend(loc="best")
ax.set_xlabel("distance from tide input creek (m)")
ax.set_ylabel("average SSC (g/m3)")
ax.set_xlim(0,1000)
ax.text(700, 27.5, str('$Ucd$ = %.2f m/s\n$ws$ = %.2f mm/s' % (egm.Ucd, egm.ws*1000)), va='center', ha='center')
pyplot.show()

# STEP 4A: Testing other set of parameters
Now lets change our parameters, but mind to set *ws* as a negative value, as the automatic check is carried on only when we initialise the EGM Framework. Let's try:

- $Ucd$ = 0.01 m/s
- $ws$ = -0.1 mm/s

The good news is that we don't need to run the Hydrodynamic model this time, as it is all saved in the *db* list.

In [ ]:
egm.Ucd = 0.01   #m/s
egm.ws = -0.0001 #m/s

# Loop through the stages
for i, stage in enumerate(egm.stages):
    
    print('\n\nSimulating stage', stage)
    data = db[i]
    iteration, error = 1, 999
    
    # Re-run the STM until get a maximum difference between consecutive runs lower than 0.05 g/m3
    while error > 0.05:        
        
        if iteration > 1:
            data.ssc0 = next_ssc0
        
        #run sediment transport model
        data = egm.update_sediment_transport(stage, hfd=data)
        next_ssc0 = data.ssc[-1,:]
        
        #computing error (between ssc series this time):
        if iteration > 1:
            error = np.max(np.abs(data.ssc - previous))
            print('\nIteration %i;    Maximum Abs. Error = %.4f' % (iteration, error))
        
        previous = data.ssc
        iteration += 1

#computing average concentration and plotting it
fig, ax = pyplot.subplots(dpi=100, figsize=(8,4), constrained_layout=True)
for i in range(egm.Nstages):
    #STM average concentration
    C = np.average(db[i].ssc, axis=0, weights=db[i].h)
    #plotting
    p = ax.plot(egm.x[2:], C[2:], label=egm.stages[i])
    ax.plot(egm.x[2:], Cbt[i][2:], ':', color=p[-1].get_color())
    
ax.grid()
ax.legend(loc="best")
ax.set_xlabel("distance from tide input creek (m)")
ax.set_ylabel("average SSC (g/m3)")
ax.set_xlim(0,1000)
ax.text(700, 27.5, str('$Ucd$ = %.2f m/s\n$ws$ = %.2f mm/s' % (egm.Ucd, egm.ws*1000)), va='center', ha='center')
pyplot.show()

# STEP 4B: Change surface roughness
The original data from `example_domain2` applied a Manning's roughness coefficient of 0.30 all over the floodplain. But, as the *Bathtub* approach assumes no flow attenuation, we can try to mimic this behaviour by reducing the roughness to a much lower value and re-run the models. That is not a big trouble. What we need to do is:
1. Manually update the roughness-parameter in all links
2. Run the Hydrodynamic model for each stage and replace the new results in the *db* list
3. Go back to **STEP 4A** and run the STM again

To tackle task one we can make use of two function from the **egmlib**: *parameters_update()* or *define_parameters()*. We used this last one in `Tutorial 1` and `example_domain2`. It requires a long list of domain/links properties and a matrix with *Manning's n* on each cell. On the other hand, we can call the first function passing only a new *egm.params*  matrix, which for our very simple domain here is easy to modify.


We already have some code to run the Hydrodynamic model until converge to a stable solution. Thus, let's do it!

In [ ]:
# Changing roughness in the params matrix. As all links are land-to-land type, we just need to set the third
#index of all links to the new coefficient, let's say: n = 0.005
print('Sample of parameters before change:\n', egm.params[0:4,0:3])
egm.params[:,2] = 0.005
print('\nSample of parameters after change:\n', egm.params[0:4,0:3])
parameters_update(params=egm.params)

# Re-run the Hydrodynamic model
db, t0 = [], datetime.now()
for stage in egm.stages:
    
    data = None
    error, iteration = 999, 1
    print('\n\nSimulating stage', stage)
    
    while error > 0.0005:
        egm.initialize_hydrodynamic(stage, hfd=data)
        data = egm.update_hydrodynamic(returnType='highFreqData')
        
        if iteration > 1:
            error = np.max(np.abs(data.h - previous))
            print('\nIteration %i;    Maximum Abs. Error = %.4f' % (iteration, error))
        
        previous = data.h
        iteration += 1
        
    db.append(data)
print('\nElapsed time:', (datetime.now() - t0))

# Now, go back to STEP 4A and run its code again to see the results

# The End
In this tutorial we managed to couple the hydrodynamic and sediment transport model using the structure provided by the **egmlib** library.

We were able to run the STM model under different conditions and for different input series.

Some plots were created to check the time evolution of SSC series, but mainly the average SSC profile, which will be used as input for some EGM models as you will see in `Tutorial 4` and `Tutorial 5`.